# Developing script: analyse_nodes

# definitions

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle

from collections import OrderedDict

import matplotlib.pyplot as plt
import seaborn as sns

# modified visualpriors library
from transforms import VisualPriorRepresentation

from classes_analyse_nodes import (
    ImageDataset,
    Pattern_Generator,
    Activation_Pattern,
    #NetworkScorer,
    calculate_integration_coeff,
    taskonomy_activation_layer_shapes,
    )

import torch
import torch.utils.model_zoo # required to load nets
from torchvision.models.feature_extraction import get_graph_node_names, create_feature_extractor

from scipy.stats import spearmanr

In [2]:
DATASET_NAMES = ('places1', 'places2', 'oasis')
SCALE_NAMES = ('scale2','scale4','scale8','scale16','scale32')

In [3]:
DATA_PATH = './data_256x256'
BEHAVIOR_PATH = './behavior'
RESULTS_PATH = './results_taskonomy'

In [4]:
#VisualPrior.viable_feature_tasks
MODEL_NAMES = ('autoencoding','depth_euclidean','jigsaw','reshading',
               'edge_occlusion','keypoints2d','room_layout', #'colorization' currently not working
               'curvature','edge_texture','keypoints3d','segment_unsup2d',
               'class_object','egomotion','nonfixated_pose','segment_unsup25d',
               'class_scene','fixated_pose','normal','segment_semantic',
               'denoising','inpainting','point_matching','vanishing_point')

In [5]:
IMAGE_TRANSFORMS = ('untransformed')

In [6]:
# storing a NetworkScorer object for saving backprojected scores
# TODO needs to be adapted when analysing multiple nets
BACKPROJECTED_SCORES_FOLDER = './backprojected_scores'

# Integration for node subsets

In [8]:
#setup net
model_name = MODEL_NAMES[18] #segment semantic
VisualPriorRepresentation._load_unloaded_nets([model_name])
net = VisualPriorRepresentation.feature_task_to_net[model_name]

In [9]:
#setup activation extractor
_, eval_nodes = get_graph_node_names(net)
return_nodes = { node:node for node in eval_nodes if "conv" in node or 'fc' in node}
activation_extractor = create_feature_extractor(net, return_nodes=return_nodes)

In [10]:
# get shape of activation
dataset = ImageDataset(os.path.join(DATA_PATH, DATASET_NAMES[0], SCALE_NAMES[0]))
dummy_image = next(iter(dataset))
net_activation = activation_extractor(dummy_image[0])
net_activation = OrderedDict(net_activation)

activation_shapes = taskonomy_activation_layer_shapes(net_activation)

In [10]:
num_subsets = 10 #10 000
num_layers = activation_shapes.__len__()
num_images = dataset.img_count

pat = Pattern_Generator(
    num_subsets,
    activation_shapes,
    frac=0.01
    )

# layer x image x subset
integration = np.full([num_layers,num_images,num_subsets], np.nan)
integration.shape

(49, 250, 10)

In [11]:
cnt = 0

# iterate image set
for img_id, (img_full, img_v1, img_v2) in enumerate(iter(dataset)):
    if cnt % 30 == 0: print(cnt, end= ' ')
    cnt=cnt+1
    
    act_full = Activation_Pattern(activation_extractor(img_full))
    act_v1 = Activation_Pattern(activation_extractor(img_v1))
    act_v2 = Activation_Pattern(activation_extractor(img_v2))
    act_avg = Activation_Pattern.average(act_v1, act_v2)

    # iterate node subsets
    for subset_num in range(num_subsets):
        subset_mask = pat.get_subset_pattern(subset_num)
        
        subset_act_full = act_full[subset_mask]
        subset_act_avg = act_avg[subset_mask]

        # calculate integration and store it
        subset_integration = calculate_integration_coeff(subset_act_full, subset_act_avg)
        integration[:,img_id, subset_num] = subset_integration



0 30 60 90 120 150 180 210 240 

# Score (correlate) integration-beauty

In [12]:
# load beauty ratings
beauty_ratings = ImageDataset(
    os.path.join(DATA_PATH, DATASET_NAMES[0], SCALE_NAMES[0]),
    beauty_ratings_path='behavior/ratings_study1.csv').beauty_ratings

In [13]:
def correlate_integration_beauty(integration: np.ndarray, beauty_ratings: pd.Series):
    return np.apply_along_axis(lambda c: spearmanr(c, beauty_ratings)[0], 1, integration)

In [14]:
correlations = correlate_integration_beauty(integration, beauty_ratings)
scores = np.abs(correlations)
scores.shape

(49, 10)

# map back scores onto nodes

In [15]:
# layer x subset
scores.shape, type(scores)

((49, 10), numpy.ndarray)

In [16]:
ns = NetworkScorer(activation_shapes)
ns.map_back_scores(scores, pat)
ns.subset_iterations_count

NameError: name 'NetworkScorer' is not defined

In [ ]:
# save NetworkScorer
ns.save(BACKPROJECTED_SCORES_FOLDER)

In [ ]:
# load NetworkScorer
loaded_ns = NetworkScorer.load('./backprojected_scores')
ns = loaded_ns
len(ns.scores.keys()), ns.subset_iterations_count

(49, 10)

In [48]:
# compare how much storing and loading image activations is to recalculating activations
# implement storing and loading image activations


IMG_ACTIVATIONS_FOLDER = './saved_image_activations'

## single image scoring

In [7]:
# storing a NetworkScorer object for saving backprojected scores
# TODO needs to be adapted when analysing multiple nets
BACKPROJECTED_SCORES_FOLDER = './backprojected_scores_img'

In [28]:
# get image with high integration rating

# calculate integration for every image in the dnn
integration = np.full([dataset.img_count,], np.nan)

for img_id, (img_full, img_v1, img_v2) in enumerate(iter(dataset)):
    act_full = Activation_Pattern(activation_extractor(img_full))
    act_v1 = Activation_Pattern(activation_extractor(img_v1))
    act_v2 = Activation_Pattern(activation_extractor(img_v2))
    act_avg = Activation_Pattern.average(act_v1, act_v2)

    i = calculate_integration_coeff(act_full, act_avg)
    integration[img_id] = i.mean()


In [31]:
highest_integration_img_idx = integration.argmax()

In [32]:
dataset.img_list[highest_integration_img_idx]

'Places365_val_00001647.mat'

In [ ]:
# get image with high integration rating

# calculate integration for every image in the dnn
integration = np.full([dataset.img_count,], np.nan)

for img_id, (img_full, img_v1, img_v2) in enumerate(iter(dataset)):
    act_full = Activation_Pattern(activation_extractor(img_full))
    act_v1 = Activation_Pattern(activation_extractor(img_v1))
    act_v2 = Activation_Pattern(activation_extractor(img_v2))
    act_avg = Activation_Pattern.average(act_v1, act_v2)

    i = calculate_integration_coeff(act_full, act_avg)
    integration[img_id] = i.mean()


In [ ]:
num_subsets = 10 #10 000
num_layers = activation_shapes.__len__()
num_images = dataset.img_count

pat = Pattern_Generator(
    num_subsets,
    activation_shapes,
    frac=0.01
    )

# layer x image x subset
integration = np.full([num_layers,num_images,num_subsets], np.nan)
integration.shape

# parallel processing

# track convergence of network scores
for now just do 10 000 iterations and already save steps

In [ ]:
# store progression on network scores, by saving NetworScorer every 10/100/200 subsets

# ideas: normalize scores and track convergence of these scores onto stable values
    # maybe need high numeric accuracy for this

# analyse influence of frac
frac represents the fraction (0, 1] of nodes in a subset of the total number of nodes in each network layer

- initially determine a reasonable frac size for a initial analysis
- later run whole analysis again for different frac sizes

Compare different choices for frac empirically, plot different subsets together and make a plot with variance bars.

With a frac of .33 , the results for the different subsets is nearly always identical, due to the law of large numbers.

(It may be possible to estimate a reasonable subset size with Tschebytschev to estimate how much variance there is with )different subset sizes.

## Visualize standard deviation

In [ ]:
scores_mean = scores.mean(axis=1)
scores_std = scores.std(axis=1)

In [ ]:
scores_mean = pd.Series(scores.mean(axis=1))
scores_std = pd.Series(scores.std(axis=1))

In [ ]:
plt.errorbar(range(49), ib_corr_mean, ib_corr_std, ecolor='red')

In [ ]:
window = 6
plt.errorbar(range(49),
             ib_corr_mean.rolling(window).mean(),
             yerr=ib_corr_std.rolling(window).mean(),
             ecolor='red')

### run analysis for different values of frac

In [ ]:
frac_values = [0.0001, 0.0005, 0.001, 0.005,  0.01, 0.05, 0.1, 0.33]

In [ ]:
def run_subset_analysis(num_subsets:int, frac: float) -> np.ndarray:
    """Run analysis correlating integration with beauty for n subsets
    with a give  subset fraction (of total network nodes).
    """
    pass

In [ ]:
def plot_subset_correlation_std_dev(results: float) -> None:
    pass
    #run analysis
    #reshape
    #plot

### variance bars for different frac sizes
visualize variance of subset ib-correlations in different layers

In [ ]:
"""Result: layer x subset matrix"""
for subset_num in range(num_subsets):
    integration = pd.DataFrame()

### look if results generalize to different frac

# dismissed: save and load images instead of recalculating activation every time

## determine if saving and loading is feasible

In [ ]:
sys.getsizeof(act_full.activation_pattern['conv1'])

80

In [ ]:
act_full.activation_pattern['conv1'].element_size() * act_full.activation_pattern['conv1'].nelement()/1E6

4.194304

In [ ]:
# calculate activation pattern size
layer_sizes = np.array([t.element_size()*t.nelement() for t in act_full.activation_pattern.values()])
print("Total size in MB:")
layer_sizes.sum()/1E6

Total size in MB:


48.627712

In [ ]:
# this makes saving and loading look possible

## test saving and loading

In [ ]:
# iterate image set
for img_id, (img_full, img_v1, img_v2) in enumerate(iter(dataset)):
    
    act_full = Activation_Pattern(activation_extractor(img_full))
    act_v1   = Activation_Pattern(activation_extractor(img_v1))
    act_v2   = Activation_Pattern(activation_extractor(img_v2))
    act_avg  = Activation_Pattern.average(act_v1, act_v2)


In [ ]:
# iterate image set
for img_id, (img_full, img_v1, img_v2) in enumerate(iter(dataset)):

    with open(os.path.join(IMG_ACTIVATIONS_FOLDER, str(img_id) + '.pkl'), 'rb') as file:
    act_full = pickle.load(file)

    with open(os.path.join(IMG_ACTIVATIONS_FOLDER, str(img_id) + '.pkl'), 'rb') as file:
    act_avg = pickle.load(file)

In [ ]:
with open(os.path.join(IMG_ACTIVATIONS_FOLDER, str(img_id) + '.pkl'), 'wb') as file:
    pickle.dump(act_full, file)

In [ ]:
with open(os.path.join(IMG_ACTIVATIONS_FOLDER, str(img_id) + '.pkl'), 'rb') as file:
    act_full_loaded = pickle.load(file)

In [ ]:
# loading works

## compare time to recalculating every time

In [ ]:
# baseline
for img_id, (img_full, img_v1, img_v2) in enumerate(iter(dataset)):
    
    act_full = Activation_Pattern(activation_extractor(img_full))
    act_v1   = Activation_Pattern(activation_extractor(img_v1))
    act_v2   = Activation_Pattern(activation_extractor(img_v2))
    act_avg  = Activation_Pattern.average(act_v1, act_v2)


In [ ]:
# new
for img_id in range(250):

    with open(os.path.join(IMG_ACTIVATIONS_FOLDER, str(249) + '.pkl'), 'rb') as file:
        act_full = pickle.load(file)

    with open(os.path.join(IMG_ACTIVATIONS_FOLDER, str(img_id) + '.pkl'), 'rb') as file:
        act_avg = pickle.load(file)